# Criando um AgentExecutor com memória

Para criarmos uma aplicação funcional de chat com nosso aggent, é necessário que ele tenha a capacidade de armazenar as informações trocadas com o usuário. E para isso, adicionamos o componente de memory já visto anteriormente no curso de Aplicações de IA com LangChain.

In [1]:
from dotenv import load_dotenv, find_dotenv
_= load_dotenv(find_dotenv())

## Criando as tools que usaremos

In [16]:
from langchain.agents import tool
from pydantic import BaseModel, Field
import datetime
import wikipedia
import requests

class ArgsTemp(BaseModel):
    latitude: float = Field(description='Latitude da localidade que buscamos a temperatura')
    longitude: float = Field(description='Longitude da localidade que buscamos a temperatura')

@tool
def busca_temperatura_atual(latitude: float, longitude: float):
    """Retorna a temperatura atual de uma determinada localidade"""
    URL = 'https://api.open-meteo.com/v1/forecast'

    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1
    }

    resposta = requests.get(URL, params=params)
    if resposta.status_code == 200:
        resultado = resposta.json()
        hora_agora = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
        lista_horas = [datetime.datetime.fromisoformat(temp_str) for temp_str in resultado['hourly']['time']]
        index_mais_prox = min(range(len(lista_horas)), key=lambda x: abs(lista_horas[x]-hora_agora))
        temp_atual = resultado['hourly']['temperature_2m'][index_mais_prox]
        return f"{temp_atual}°C"
    else:
        raise Exception(f'Request para a API {URL} falhou: {resposta.status_code}')
    
wikipedia.set_lang('pt')

@tool
def busca_wikipedia(query: str):
    """Faz uma busca na Wikipédia de uma query e retorna resumos de páginas para a query"""
    titulos_paginas = wikipedia.search(query)
    resumos = []
    for titulo in titulos_paginas[:3]:
        try:
            wiki_page = wikipedia.page(title=titulo, auto_suggest=True)
            resumos.append(f'Título da página: {titulo} \nResumo: {wiki_page.summary}')
        except:
            pass
    if not resumos:
        return "Busca não teve retorno"
    else:
        return '\n\n'.join(resumos)

## O que temos no final?

In [17]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.schema.agent import AgentFinish
from langchain.schema.runnable import RunnablePassthrough

chat = ChatOpenAI(model='gpt-4o-mini')

tools = [busca_temperatura_atual, busca_wikipedia]
tools_json = [convert_to_openai_function(tool)for tool in tools]
tools_run = {tool.name: tool for tool in tools}

### Um Agent

In [18]:
prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável e gentil chamado Mimir'),
    ('user', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])

pass_through = RunnablePassthrough.assign(
    agent_scratchpad = lambda x: format_to_openai_function_messages(x['intermediate_steps'])
)

agent_chain = pass_through | prompt | chat.bind(functions=tools_json) |  OpenAIFunctionsAgentOutputParser()

### Um AgentExecutor

In [19]:
def run_agent(input):
    passos_intermediarios = []
    while True:
        resposta = agent_chain.invoke({
            'input': input,
            'intermediate_steps': passos_intermediarios
            })
        if isinstance(resposta, AgentFinish):
            return resposta
        observacao = tools_run[resposta.tool].run(resposta.tool_input)
        passos_intermediarios.append((resposta, observacao))

In [20]:
resposta = run_agent('Qual a temperatura de Floripa?')
resposta

AgentFinish(return_values={'output': 'A temperatura atual em Florianópolis é de 17,1°C.'}, log='A temperatura atual em Florianópolis é de 17,1°C.')

In [21]:
resposta = run_agent('Meu nome é Thomas')
resposta

AgentFinish(return_values={'output': 'Olá, Thomas! Como posso ajudá-lo hoje?'}, log='Olá, Thomas! Como posso ajudá-lo hoje?')

In [22]:
resposta = run_agent('Qual é o meu nome?')
resposta

AgentFinish(return_values={'output': 'Desculpe, mas não tenho acesso a informações pessoais, incluindo o seu nome. Mas estou aqui para ajudar com qualquer outra coisa que você precise!'}, log='Desculpe, mas não tenho acesso a informações pessoais, incluindo o seu nome. Mas estou aqui para ajudar com qualquer outra coisa que você precise!')

## LangChain AgentExecutor

O LangChain fornece uma estrutura pronta de AgentExecutors. Além de já fazer o loop necessário para processamento das mensagens trocadas com o modelo, ele permite algumas outras funcionalidades como verbose=True que facilita o debugar e a adição de memórias que veremos a seguir:

In [23]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent_chain,
    tools=tools,
    verbose=True
)

In [25]:
resposta = agent_executor.invoke({'input':'Qual a temperatura de Floripa?'})
resposta



> Entering new AgentExecutor chain...

Invoking: `busca_temperatura_atual` with `{'latitude': -27.5956, 'longitude': -48.548}`


17.1°CA temperatura atual em Florianópolis é de 17,1°C. Se precisar de mais alguma coisa, estou à disposição!

> Finished chain.


{'input': 'Qual a temperatura de Floripa?',
 'output': 'A temperatura atual em Florianópolis é de 17,1°C. Se precisar de mais alguma coisa, estou à disposição!'}

In [26]:
resposta = agent_executor.invoke({'input':'Olá'})
resposta



> Entering new AgentExecutor chain...
Olá! Como posso ajudá-lo hoje? 😊

> Finished chain.


{'input': 'Olá', 'output': 'Olá! Como posso ajudá-lo hoje? 😊'}

## Adicionando memória

Vamos usar uma memória bem simples neste caso e mostrar as modificações necessárias no prompt e no AgentExecutor:

In [34]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history'
)

In [29]:
prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável e gentil chamado Mimir'),
    MessagesPlaceholder(variable_name='chat_history'),
    ('user', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])

pass_through = RunnablePassthrough.assign(
    agent_scratchpad = lambda x: format_to_openai_function_messages(x['intermediate_steps'])
)

agent_chain = pass_through | prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

In [30]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent_chain,
    tools=tools,
    memory=memory,
    verbose=True
)

In [31]:
resposta = agent_executor({'input': 'Olá, meu nome é Thomas'})
resposta

C:\Users\maris\AppData\Local\Temp\ipykernel_3652\2960953260.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  resposta = agent_executor({'input': 'Olá, meu nome é Thomas'})




> Entering new AgentExecutor chain...
Olá, Thomas! Como posso ajudar você hoje? 😊

> Finished chain.


{'input': 'Olá, meu nome é Thomas',
 'chat_history': [HumanMessage(content='Olá, meu nome é Thomas', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Olá, Thomas! Como posso ajudar você hoje? 😊', additional_kwargs={}, response_metadata={})],
 'output': 'Olá, Thomas! Como posso ajudar você hoje? 😊'}

In [32]:
resposta = agent_executor({'input': 'Qual é o meu nome?'})
resposta



> Entering new AgentExecutor chain...
Seu nome é Thomas! Como posso ajudar você hoje, Thomas?

> Finished chain.


{'input': 'Qual é o meu nome?',
 'chat_history': [HumanMessage(content='Olá, meu nome é Thomas', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Olá, Thomas! Como posso ajudar você hoje? 😊', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Qual é o meu nome?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Seu nome é Thomas! Como posso ajudar você hoje, Thomas?', additional_kwargs={}, response_metadata={})],
 'output': 'Seu nome é Thomas! Como posso ajudar você hoje, Thomas?'}